In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import chardet
import os
import numpy as np

In [3]:
set_diff_df=[]
val=[]
name_list=pd.read_excel('/content/drive/My Drive/Attendance/namelist.xlsx')
name_list['Roll Number']=name_list['Roll Number'].str.strip()
reg_no_full=name_list['Roll Number']
path='/content/drive/My Drive/Attendance/data/'
os.chdir('/content/drive/My Drive/Attendance/data')
for file in os.listdir():
  if file.endswith(".csv"):
    print(file)
    with open(path+file, 'rb') as f:
              result = chardet.detect(f.read())
    data = pd.read_csv(path+file,encoding=result['encoding'],sep='\t',skiprows = 6) 

    
    data['Reg No']=data['Full Name'].apply(lambda st: st[st.find("[")+1:st.find("]")])
    data["Join Time"] = data["Join Time"].apply(pd.to_datetime)

    new = data["Duration"].str.split(" ", n = 1, expand = True)
    data["T1"]= new[0]
    data["T2"]= new[1]

    data["T1"].fillna("0s",inplace=True)
    data["T2"].fillna("0s",inplace=True)

    data['Activity_1']=0
    data['Activity_2']=0

    # create a list of our conditions
    conditions = [(data['T1'].str.contains('h')),(data['T1'].str.contains('m')),(data['T1'].str.contains('s'))]

    # create a list of the values we want to assign for each condition
    values = [(data['T1'].str.extract('(\d+)').astype(float)), (data['T1'].str.extract('(\d+)').astype(float)), ((data['T1'].str.extract('(\d+)').astype(float)))]

    # create a new column and use np.select to assign values to it using our lists as arguments
    data['Activity_1']  = np.select(conditions, values)

    conditions = [(data['T1'].str.contains('h')),(data['T1'].str.contains('m')),(data['T1'].str.contains('s')) ]

    values = [data['Activity_1'] *3600, data['Activity_1']*60, data['Activity_1'] ]

    data['Activity_1']  = np.select(conditions, values)

    # create a list of our conditions
    conditions = [(data['T2'].str.contains('h')),(data['T2'].str.contains('m')),(data['T2'].str.contains('s'))]

    # create a list of the values we want to assign for each condition
    values = [(data['T2'].str.extract('(\d+)').astype(float)), (data['T2'].str.extract('(\d+)').astype(float)), ((data['T2'].str.extract('(\d+)').astype(float)))]

    # create a new column and use np.select to assign values to it using our lists as arguments
    data['Activity_2']  = np.select(conditions, values)

    conditions = [(data['T2'].str.contains('h')),(data['T2'].str.contains('m')),(data['T2'].str.contains('s'))]

    values = [data['Activity_2'] *3600, data['Activity_2']*60, data['Activity_2'] ]

    data['Activity_2']  = np.select(conditions, values)

    data['total_time_spent']=data['Activity_1']+data['Activity_2']
    #print(data.shape)
    final_df=data.groupby(['Reg No'],as_index = False).sum()
    #print(final_df.shape)
    final_df.drop(['Activity_1','Activity_2'],axis=1,inplace=True)
    #final_df.reset_index(inplace=True)
    #final_df.drop_duplicates(subset ="Reg No",keep ='first', inplace = True)
    final_df=final_df[final_df['total_time_spent']>1800]
    #print(final_df.shape)

    present=final_df['Reg No']
    set_diff_df.append((pd.concat([reg_no_full, present, present]).drop_duplicates(keep=False)).to_string(index=False))
    val.append(str(data['Join Time'].iloc[0].date().day)+'-'+str(data['Join Time'].iloc[0].date().month)+'-'+str(data['Join Time'].iloc[0].date().year))
new_val= pd.DataFrame(list(zip(val, set_diff_df)),columns =['date', 'absentees']) 
new_val.to_excel('/content/drive/My Drive/Attendance/data/consolidated.xlsx',index=False)

meetingAttendanceReport(19CSE102-Tuesday - Session -4).csv
meetingAttendanceReport(19CSE102-Tuesday - Session -4) (1).csv
meetingAttendanceReport(19CSE102-Monday - First Session).csv
meetingAttendanceReport(19CSE102-Monday - First Session) (1).csv
meetingAttendanceReport(19CSE102-Monday - First Session) (2).csv
meetingAttendanceReport( 19CSE102- Friday - First Session).csv
meetingAttendanceReport( 19CSE102- Friday - First Session) (1).csv
meetingAttendanceReport( 19CSE102- Friday - First Session) (2).csv
meetingAttendanceReport(19CSE102- Lab - Friday AN).csv


In [4]:
final_df

,Reg No,total_time_spent
0,CB.EN.U4ECE20001,7866.0
2,CB.EN.U4ECE20003,9213.0
3,CB.EN.U4ECE20004,9433.0
4,CB.EN.U4ECE20005,9444.0
5,CB.EN.U4ECE20006,9255.0
...,...,...
64,CB.EN.U4ECE20067,9402.0
65,CB.EN.U4ECE20068,8852.0
66,CB.EN.U4ECE20069,6720.0
67,CB.EN.U4ECE20070,9240.0


In [5]:
data.head()

,Full Name,Join Time,Leave Time,Duration,Email,Role,Reg No,T1,T2,Activity_1,Activity_2,total_time_spent
0,NEKKANTI SRI SAI VENKATA PRANEETH - [CB.EN.U4E...,2021-04-23 13:45:52,"4/23/2021, 4:48:20 PM",3h 2m,cb.en.u4ece20037@cb.students.amrita.edu,Presenter,CB.EN.U4ECE20037,3h,2m,10800.0,120.0,10920.0
1,Samudrala Santhosh Raj - [CB.EN.U4ECE20052],2021-04-23 13:48:56,"4/23/2021, 3:52:12 PM",2h 3m,cb.en.u4ece20052@cb.students.amrita.edu,Presenter,CB.EN.U4ECE20052,2h,3m,7200.0,180.0,7380.0
2,Samudrala Santhosh Raj - [CB.EN.U4ECE20052],2021-04-23 16:00:07,"4/23/2021, 4:48:20 PM",48m 12s,cb.en.u4ece20052@cb.students.amrita.edu,Presenter,CB.EN.U4ECE20052,48m,12s,2880.0,12.0,2892.0
3,SARAN S R - [CB.EN.U4ECE20053],2021-04-23 13:52:08,"4/23/2021, 4:48:18 PM",2h 56m,cb.en.u4ece20053@cb.students.amrita.edu,Presenter,CB.EN.U4ECE20053,2h,56m,7200.0,3360.0,10560.0
4,Rupika M - [CB.EN.U4ECE20050],2021-04-23 13:55:45,"4/23/2021, 3:50:59 PM",1h 55m,cb.en.u4ece20050@cb.students.amrita.edu,Presenter,CB.EN.U4ECE20050,1h,55m,3600.0,3300.0,6900.0
